In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn.metrics as sk_metrics
import tempfile
import os

2023-03-13 12:20:36.315620: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 12:20:37.016322: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 12:20:37.016343: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-13 12:20:37.108416: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-13 12:20:38.688835: W tensorflow/stream_executor/platform/de

In [2]:
# Preset matplotlib figure sizes.
matplotlib.rcParams['figure.figsize'] = [9, 6]

print(tf.__version__)
# To make the results reproducible, set the random seed value.
tf.random.set_seed(22)

2.10.0


In [3]:
dataset = pd.read_csv('training_full.csv')
dataset.drop(columns=dataset.columns[0], axis=1, inplace=True)
dataset = dataset[ ['label'] + [ col for col in dataset.columns if col != 'label' ] ]
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3572 entries, 0 to 3571
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   label           3572 non-null   bool   
 1   tconst          3572 non-null   object 
 2   primaryTitle    3572 non-null   object 
 3   originalTitle   3572 non-null   object 
 4   startYear       3572 non-null   object 
 5   endYear         3572 non-null   object 
 6   runtimeMinutes  3572 non-null   object 
 7   numVotes        3572 non-null   float64
 8   writer          3572 non-null   object 
 9   director        3572 non-null   object 
dtypes: bool(1), float64(1), object(8)
memory usage: 254.8+ KB


In [4]:
train_dataset = dataset.sample(frac=0.75, random_state=1)
test_dataset = dataset.drop(train_dataset.index)
x_train, y_train = train_dataset.iloc[:, 1:], train_dataset.iloc[:, 0]
x_test, y_test = test_dataset.iloc[:, 1:], test_dataset.iloc[:, 0]

In [5]:
x_train

,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,writer,director
2062,tt1151922,Miss March,Miss March,2009,\N,90,21245.0,"nm1199107, nm2222515, nm1380203, nm1469262, nm...","nm2222515, nm1199107"
1946,tt10309552,12 Hour Shift,12 Hour Shift,2020,\N,86,3906.0,nm2647420,nm2647420
1688,tt0445990,Invincible,Invincible,2006,\N,105,68930.0,nm1048533,nm0179697
3313,tt6615648,Manu,Manu,2018,\N,180,1060.0,nm5186373,nm5186373
1496,tt0327419,Amir El Zalam,Amir El Zalam,2002,\N,\N,1567.0,"nm1217830, nm1219749",nm1216773
...,...,...,...,...,...,...,...,...,...
1901,tt0978670,The Neighbor,The Neighbor,\N,2007,98,1170.0,"nm0204731, nm1376000",nm1376000
1975,tt1058743,Pretty Bird,Pretty Bird,2008,\N,120,1124.0,"nm0049026, nm0773973",nm0773973
776,tt0092501,Prớjéct Á 2,'A' gai wak 2,1987,\N,106,9943.0,"nm0000329, nm0849282, nm0950568",nm0000329
2652,tt2370140,Crossfire Hurricane,Crossfire Hurricane,\N,2012,111,2532.0,nm0605137,nm0605137


In [6]:
y_train, y_test = y_train.map({False: 0, True: 1}), y_test.map({False: 0, True: 1})
x_train, y_train = tf.convert_to_tensor(x_train, dtype=tf.float32), tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test, y_test = tf.convert_to_tensor(x_test, dtype=tf.float32), tf.convert_to_tensor(y_test, dtype=tf.float32)

2023-03-13 12:20:41.993822: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-13 12:20:41.994308: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-13 12:20:41.994362: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (maxim-P453UA): /proc/driver/nvidia/version does not exist
2023-03-13 12:20:41.998228: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: could not convert string to float: 'tt1151922'